## **Setting up the Environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


## **Importing Libraries**

In [3]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.utils import resample

## **Setting up Stemmer**

In [4]:
! rm -r tamil-stemmer-build
! rm input.txt
! rm output.txt

! mkdir tamil-stemmer-build
% cd tamil-stemmer-build
! git clone https://github.com/rdamodharan/tamil-stemmer.git
! tar -zxf tamil-stemmer/snowball-with-tamil.tgz
% cd snowball-with-tamil
! make

% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build
Cloning into 'tamil-stemmer'...
remote: Enumerating objects: 88, done.
remote: Total 88 (delta 0), reused 0 (delta 0), pack-reused 88
Unpacking objects: 100% (88/88), done.
/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build/snowball-with-tamil
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/space.o compiler/space.c
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/tokeniser.o compiler/tokeniser.c
In file included from compiler/tokeniser.c:6:0:
compiler/tokeniser.c: In function ‘read_token’:
compiler/header.h:8:21: warning: this statement may fall through [-Wimplicit-fallthrough=]
 #define unless(C) if(!(C))
                     ^
compiler/tokeniser.c:390:16: note: in expansion of macro ‘unless’
                unless (t->next == 0) {
               

## **Loading Data**

In [5]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [6]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [7]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam1,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [8]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [9]:
dataframe.shape

(5880, 3)

In [10]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [11]:
dataframe.shape

(5877, 3)

In [12]:
text = dataframe['text']

In [13]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [14]:
label = dataframe['category']

In [15]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [16]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [17]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

In [18]:
text = text.tolist()

! rm input.txt
! rm output.txt

for i in text:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

rm: cannot remove 'input.txt': No such file or directory
rm: cannot remove 'output.txt': No such file or directory


In [19]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [20]:
text_file = open("output.txt", "r")

text = []

for line in text_file:
    text.append(line.strip())

In [21]:
text = np.array(text)
text

array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்',
       'த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்', ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype='<U795')

## **Feature Extraction**

In [22]:
vectorizer = TfidfVectorizer(min_df = 5)
X = vectorizer.fit_transform(text)
X = X.todense()

In [23]:
X.shape

(5877, 782)

In [24]:
y = label
y.shape

(5877,)

## **Down Sampling**

In [25]:
df = pd.DataFrame(zip(X, y))
df_0 = df[df[1] == 0]
df_1 = df[df[1] == 1]
print(df_0.shape)
print(df_1.shape)

(4724, 2)
(1153, 2)


In [26]:
df_0_downsample = resample(df_0, replace=True, n_samples=len(df_1), random_state=42)
print(df_0_downsample.shape)
print(df_1.shape)

(1153, 2)
(1153, 2)


In [27]:
df_new = pd.concat([df_0_downsample, df_1])
X = df_new[0].values.tolist()
y = df_new[1].values.tolist()
X = np.array(X)
y = np.array(y)

## **Train Test Split**

In [32]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[2])
X_val = X_val.reshape(X_val.shape[0], X_val.shape[2])

In [33]:
X_train.shape

(1844, 782)

In [34]:
X_val.shape

(462, 782)

## **Logistic Regression**

In [35]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.75      0.71      0.73       228
           1       0.73      0.76      0.75       234

    accuracy                           0.74       462
   macro avg       0.74      0.74      0.74       462
weighted avg       0.74      0.74      0.74       462



## **Support Vector Machine**

In [36]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.79      0.67      0.73       228
           1       0.72      0.82      0.77       234

    accuracy                           0.75       462
   macro avg       0.75      0.75      0.75       462
weighted avg       0.75      0.75      0.75       462



## **Naive Bayes**

In [37]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.57      0.79      0.66       228
           1       0.67      0.42      0.52       234

    accuracy                           0.60       462
   macro avg       0.62      0.61      0.59       462
weighted avg       0.62      0.60      0.59       462



## **Stochastic Gradient Descent**

In [38]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.73      0.68      0.70       228
           1       0.71      0.75      0.73       234

    accuracy                           0.72       462
   macro avg       0.72      0.72      0.72       462
weighted avg       0.72      0.72      0.72       462



## **K Nearest Neighbours**

In [39]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.56      0.90      0.69       228
           1       0.76      0.31      0.44       234

    accuracy                           0.60       462
   macro avg       0.66      0.60      0.56       462
weighted avg       0.66      0.60      0.56       462



## **Decision Tree**

In [40]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.67      0.71      0.69       228
           1       0.70      0.66      0.68       234

    accuracy                           0.69       462
   macro avg       0.69      0.69      0.69       462
weighted avg       0.69      0.69      0.69       462



## **Random Forest**

In [41]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.73      0.71      0.72       228
           1       0.73      0.75      0.74       234

    accuracy                           0.73       462
   macro avg       0.73      0.73      0.73       462
weighted avg       0.73      0.73      0.73       462



## **Majority Voting**

In [42]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Removing KNN and NB from predictions due to poor performance
    predictions = [lr_pred[i], svm_pred[i], sgd_pred[i], 
                   dt_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))

              precision    recall  f1-score   support

           0       0.76      0.71      0.74       228
           1       0.74      0.79      0.76       234

    accuracy                           0.75       462
   macro avg       0.75      0.75      0.75       462
weighted avg       0.75      0.75      0.75       462



## **Loading Testing Data**

In [43]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [44]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
freq,1,1


In [45]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [46]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [47]:
text_test = text_test.tolist()

! rm input.txt
! rm output.txt

for i in text_test:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

In [48]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [49]:
text_file = open("output.txt", "r")

text_test = []

for line in text_file:
    text_test.append(line.strip())

text_test = np.array(text_test)

## **Test Feature Extraction**

In [50]:
vectorizer = TfidfVectorizer(min_df = 5)
X_test = vectorizer.fit_transform(text_test)
X_test = X_test.todense()

In [51]:
X_test.shape

(654, 237)

In [52]:
X_test = X_test.tolist()

In [53]:
for i in range(len(X_test)):
    for n in range(545):
        X_test[i].append(0)

In [54]:
X_test = np.array(X_test)
X_test.shape

(654, 782)

## **Getting Predictions**

In [55]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [56]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    predictions = [lr_test_pred[i], svm_test_pred[i], sgd_test_pred[i], 
                   dt_test_pred[i], rf_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [57]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [58]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [59]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.84      0.55      0.67       536
           1       0.21      0.53      0.30       118

    accuracy                           0.55       654
   macro avg       0.52      0.54      0.48       654
weighted avg       0.73      0.55      0.60       654



In [60]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

Support Vector Machine
              precision    recall  f1-score   support

           0       0.86      0.49      0.62       536
           1       0.21      0.63      0.32       118

    accuracy                           0.51       654
   macro avg       0.53      0.56      0.47       654
weighted avg       0.74      0.51      0.57       654



In [61]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.86      0.39      0.54       536
           1       0.20      0.70      0.31       118

    accuracy                           0.45       654
   macro avg       0.53      0.55      0.43       654
weighted avg       0.74      0.45      0.50       654



In [62]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

Stochastic Gradient Descent
              precision    recall  f1-score   support

           0       0.82      0.56      0.67       536
           1       0.19      0.46      0.26       118

    accuracy                           0.54       654
   macro avg       0.51      0.51      0.47       654
weighted avg       0.71      0.54      0.59       654



In [63]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           0       0.83      0.82      0.83       536
           1       0.22      0.22      0.22       118

    accuracy                           0.72       654
   macro avg       0.52      0.52      0.52       654
weighted avg       0.72      0.72      0.72       654



In [64]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

Decision Tree
              precision    recall  f1-score   support

           0       0.84      0.83      0.84       536
           1       0.26      0.27      0.27       118

    accuracy                           0.73       654
   macro avg       0.55      0.55      0.55       654
weighted avg       0.73      0.73      0.73       654



In [65]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

Random Forest
              precision    recall  f1-score   support

           0       0.83      0.92      0.87       536
           1       0.29      0.14      0.19       118

    accuracy                           0.78       654
   macro avg       0.56      0.53      0.53       654
weighted avg       0.73      0.78      0.75       654



In [66]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.83      0.60      0.70       536
           1       0.20      0.45      0.28       118

    accuracy                           0.57       654
   macro avg       0.52      0.53      0.49       654
weighted avg       0.72      0.57      0.62       654



## **Saving the Predictions**

In [67]:
# Logistic Regression

predictions = le.inverse_transform(lr_test_pred)

dataframe_test_lr = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_lr = pd.DataFrame(dataframe_test_lr)
dataframe_test_lr.to_csv("Output/TF-IDF-LogisticRegression.csv")
dataframe_test_lr

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,OFF
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,OFF
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,OFF
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,OFF


In [68]:
# Support Vector Machine

predictions = le.inverse_transform(svm_test_pred)

dataframe_test_svm = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_svm = pd.DataFrame(dataframe_test_svm)
dataframe_test_svm.to_csv("Output/TF-IDF-SupportVectorMachine.csv")
dataframe_test_svm

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,OFF
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,OFF
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,OFF
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",OFF
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,OFF


In [69]:
# Naive Bayes

predictions = le.inverse_transform(nb_test_pred)

dataframe_test_nb = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_nb = pd.DataFrame(dataframe_test_nb)
dataframe_test_nb.to_csv("Output/TF-IDF-NaiveBayes.csv")
dataframe_test_nb

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,OFF
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,OFF
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,OFF
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,OFF
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,OFF
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",OFF
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,OFF


In [70]:
# Stochastic Gradient Descent

predictions = le.inverse_transform(sgd_test_pred)

dataframe_test_sgd = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_sgd = pd.DataFrame(dataframe_test_sgd)
dataframe_test_sgd.to_csv("Output/TF-IDF-StochasticGradientDescent.csv")
dataframe_test_sgd

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,OFF
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,OFF
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,OFF
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,OFF


In [71]:
# K Nearest Neighbours

predictions = le.inverse_transform(knn_test_pred)

dataframe_test_knn = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_knn = pd.DataFrame(dataframe_test_knn)
dataframe_test_knn.to_csv("Output/TF-IDF-KNearestNeighbours.csv")
dataframe_test_knn

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,OFF


In [72]:
# Decision Tree

predictions = le.inverse_transform(dt_test_pred)

dataframe_test_dt = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_dt = pd.DataFrame(dataframe_test_dt)
dataframe_test_dt.to_csv("Output/TF-IDF-DecisionTree.csv")
dataframe_test_dt

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,OFF
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [73]:
# Random Forest

predictions = le.inverse_transform(rf_test_pred)

dataframe_test_rf = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_rf = pd.DataFrame(dataframe_test_rf)
dataframe_test_rf.to_csv("Output/TF-IDF-RandomForest.csv")
dataframe_test_rf

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [74]:
# Majority Voting

predictions = le.inverse_transform(mv_test_pred)

dataframe_test_mv = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_mv = pd.DataFrame(dataframe_test_mv)
dataframe_test_mv.to_csv("Output/TF-IDF-MajorityVoting.csv")
dataframe_test_mv

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,OFF
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,OFF
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,OFF
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,OFF
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,OFF
